In [71]:
# importacion general de librerias y de visualizacion (matplotlib y seaborn)
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [72]:
eventos_2018 = pd.read_csv('events.csv')

/home/bauticavanese/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


# Localidades

In [73]:
# import galeria nesesaria para graficos de mapas.
############################################################
#          https://anaconda.org/conda-forge/folium         #
############################################################

import folium

In [74]:
# Eventos sin registro de pais de origen.
eventos_2018['country'].isnull().sum()

923910

Si hacemos la cuenta, vemos que hay aproximadamente un 10% de los datos que no son nulos, es nescesario saber esto antes
de sacar conlusiones.

## Paises ##

In [75]:
# Total de paises dentro del archivo.
countries_name = pd.read_json('data/world-countries.json')
countries_name = countries_name.features.apply(lambda x: x['properties']['name']).to_frame()
countries_name['count'] = 0
countries_name.columns = ['country', 'count']

# Calculando la cantidad de eventos por pais.
countries_data = pd.DataFrame(eventos_2018['country'].value_counts().astype(int))
countries_data.reset_index(inplace=True)
countries_data.columns = ['country', 'count']
countries_data.replace({'United States' : 'United States of America'}, inplace= True)

# Cantiadad de eventos sobre todos los paises.
data = countries_data.merge(countries_name, how='right', on='country')
data.replace({np.nan : 0}, inplace= True)

In [76]:
# Top 5 paises con mas eventos.
data[:5]

,country,count_x,count_y
0,Brazil,84308.0,0
1,United States of America,465.0,0
2,Argentina,94.0,0
3,Canada,35.0,0
4,Netherlands,13.0,0


In [77]:
# Creacion de mapa
SF_COORDINATES = (37.76, -122.45)
mapa = folium.Map(location=SF_COORDINATES, tiles="Mapbox Bright", zoom_start=2)


In [78]:
# Pintamos 
mapa.choropleth(
    geo_data='world-countries.json',
    name='Cantidad de Eventos',
    data=data,
    columns=['country', 'count_x'],
    key_on='feature.properties.name',
    fill_color='YlOrRd',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Cantidad de Eventos',
    highlight=True
)

# Decrementar para visualizar el mapa.
#display(mapa)

In [79]:
# Escala logaritmica para visualizar mejor los datos.
data['count_x_log'] = np.log(data['count_x'] + 1)

In [80]:
mapa.choropleth(
    geo_data='data/world-countries.json',
    name='Cantidad de Eventos (Log)',
    data=data,
    columns=['country', 'count_x_log'],
    key_on='feature.properties.name',
    fill_color='PuBuGn',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Cantidad de Eventos (Log)',
    highlight=True,
)


# Agregamos un controlador de layers.
folium.LayerControl().add_to(mapa)

# Decrementar para visualizar el mapa.
#display(mapa)

In [81]:
# Decrementar para visualizar el mapa.
mapa.save('country_map.html')
#display(mapa)

## Regiones

In [82]:
#Top 10 regiones que realizaron eventos.
eventos_2018['region'].value_counts()[:10]

Sao Paulo            24996
Unknown              14772
Minas Gerais          7755
Rio de Janeiro        6913
Bahia                 5737
Pernambuco            2962
Ceara                 2829
Parana                2602
Rio Grande do Sul     2576
Federal District      1739
Name: region, dtype: int64

Como Brasil es el pais en donde mas ocurrieron eventos, realizaremos un analisis mas profundo acerca de las regiones mas participes de la platafroma.

In [83]:
# Cantidad de eventos por region (Brasil)
bra_data = eventos_2018.loc[eventos_2018['country'] == 'Brazil']['region'].value_counts().to_frame()
bra_data.reset_index(inplace=True)
bra_data.columns = ['region', 'count']

In [90]:
# Total de paises dentro del archivo.
bra_regiones = pd.read_json('data/brazil-states.geojson')
bra_regiones = bra_regiones.features.apply(lambda x: x['properties']['name']).to_frame()
bra_regiones['count'] = 0
bra_regiones.columns = ['region', 'count']

# Respetando notacion de archivo json.
bra_data.replace({'Parana' : 'Paraná', 'Sao Paulo' :'São Paulo', 'Ceara' : 'Ceará', 'Piaui' : 'Piauí',
                  'Rondonia' : 'Rondônia', 'Para': 'Pará', 'Maranhao' : 'Maranhão'}, 
                 inplace=True)

# Cantiadad de eventos sobre todas las regiones.
data = bra_data.merge(bra_regiones, how='right', on='region')
data.replace({np.nan : 0}, inplace= True)

In [85]:
# Top 10 regiones en Brasil
data[:10]

,region,count_x,count_y
0,São Paulo,24996.0,0
1,Minas Gerais,7755.0,0
2,Rio de Janeiro,6913.0,0
3,Bahia,5737.0,0
4,Pernambuco,2962.0,0
5,Ceará,2829.0,0
6,Paraná,2602.0,0
7,Rio Grande do Sul,2576.0,0
8,Santa Catarina,1511.0,0
9,Pará,1382.0,0


In [86]:
# Creacion de mapa
SF_COORDINATES = (-23.533773, -46.625290)
mapa_bra = folium.Map(location=SF_COORDINATES, zoom_start=7)

mapa_bra.choropleth(
    geo_data='data/brazil-states.geojson',
    name='Cantidad de Eventos',
    data=data,
    columns=['region', 'count_x'],
    key_on='feature.properties.name',
    fill_color='PuBuGn',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Cantidad de Eventos',
    highlight=True,
    reset=True
)

#display(mapa_bra)

In [87]:
# Escala logaritmica
data['count_log'] = np.log(data['count_x']+1)

In [88]:
mapa_bra.choropleth(
    geo_data='data/brazil-states.geojson',
    name='Cantidad de Eventos LOG',
    data=data,
    columns=['region', 'count_log'],
    key_on='feature.properties.name',
    fill_color='PuBuGn',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Cantidad de Eventos LOG',
    highlight=True,
    reset=True
)

# Agregamos un controlador de layers.
folium.LayerControl().add_to(mapa_bra)

mapa_bra.save('bra_map.html')
